In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.v2 as transforms_v2  # CutMix를 위한 v2 transforms 추가
import sys
import os
import time
import random
import numpy as np
import wandb
from tqdm import tqdm
from tools.tool import AccuracyEarlyStopping, WarmUpLR, SAM, PSKD  # 수정된 AccuracyEarlyStopping 클래스 임포트
from models.resnet import resnet18

wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")
wandb.init(project="PBL-2", name="resnet18_cfc,lr=0.005,factor=0.5,SAM_SGD,PSKD")  

# WandB 설정 - ResNeXt 코드의 파라미터 사용하되 모델만 resnet18로 변경
config = {
    # 모델 설정
    "model": "resnet18",  # 모델만 resnet18로 유지
    "batch_size": 128,    
    "num_epochs": 300,
    
    "learning_rate": 0.005,  
    "optimizer": "SAM_SGD",
    "momentum": 0.9,
    "weight_decay": 5e-4,
    "nesterov": True,
    
    # SAM 옵티마이저 설정
    "rho": 0.05,
    "adaptive": False,
    
    # 학습 과정 설정
    "seed": 2025,
    "deterministic": False,
    "patience": 30,  # early stopping patience
    "max_epochs_wait": float('inf'),
    
    # 데이터 증강 설정
    "cutmix_alpha": 1.0,  # CutMix 알파 파라미터
    "cutmix_prob": 0.5,   # CutMix 적용 확률
    "crop_padding": 4,    # RandomCrop 패딩 크기
    "crop_size": 32,      # RandomCrop 크기 (CIFAR-100 이미지 크기는 32x32)
    
    # 스케줄러 설정 - ResNeXt 파라미터 적용
    "warmup_epochs": 10,  # ResNeXt와 동일하게 설정
    "lr_scheduler": "ReduceLROnPlateau",
    "lr_factor": 0.5,
    "lr_patience": 5,
    "lr_threshold": 0.1,
    "min_lr": 1e-6,
    
    # 시스템 설정
    "num_workers": 32,
    "pin_memory": True,
}

wandb.config.update(config)

# CIFAR-100 데이터셋 로드 - 기본 train/test 분할 사용
transform_train = transforms.Compose([
    transforms.RandomCrop(config["crop_size"], padding=config["crop_padding"]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

# DataLoader 생성 - config 사용
trainloader = DataLoader(
    trainset, 
    batch_size=config["batch_size"], 
    shuffle=True, 
    pin_memory=config["pin_memory"], 
    num_workers=config["num_workers"]
)

testloader = DataLoader(
    testset, 
    batch_size=config["batch_size"], 
    shuffle=False, 
    pin_memory=config["pin_memory"], 
    num_workers=config["num_workers"]
)

print(f"Train set size: {len(trainset)}")
print(f"Test set size: {len(testset)}")

# CutMix 변환 정의
cutmix = transforms_v2.CutMix(alpha=config["cutmix_alpha"], num_classes=100)  # CIFAR-100은 100개 클래스

def train(model, trainloader, criterion, optimizer, device, epoch, warmup_scheduler=None, warmup_epochs=None):
    """
    학습 함수 (CutMix 및 PSKD 적용)
    """
    # config에서 warmup_epochs 가져오기 (None이면)
    if warmup_epochs is None:
        warmup_epochs = config["warmup_epochs"]
        
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # CutMix 확률적 적용
        if random.random() < config["cutmix_prob"]:
            inputs, labels = cutmix(inputs, labels)
            # 이 경우 labels은 원-핫 인코딩 형태로 변환됨
            use_cutmix = True
        else:
            # 일반 레이블을 원-핫 인코딩으로 변환 (PSKD에 필요)
            batch_size = labels.size(0)
            one_hot_labels = torch.zeros(batch_size, 100).to(device)  # CIFAR-100은 100개 클래스
            one_hot_labels.scatter_(1, labels.unsqueeze(1), 1)
            use_cutmix = False
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        # SAM 첫 번째 스텝을 위한 손실 계산
        if use_cutmix:
            # CutMix가 적용된 경우 (이미 원-핫 인코딩된 레이블)
            loss = criterion(outputs, labels)
        else:
            # 일반적인 경우 (원-핫 인코딩으로 변환한 레이블)
            loss = criterion(outputs, one_hot_labels)
            
        loss.backward()
        optimizer.first_step(zero_grad=True)
        
        # SAM 두 번째 스텝을 위한 손실 계산
        outputs = model(inputs)
        if use_cutmix:
            loss = criterion(outputs, labels)
        else:
            loss = criterion(outputs, one_hot_labels)
        
        loss.backward()
        optimizer.second_step(zero_grad=True)
        
        # 학습률 스케줄러 업데이트 - warmup 스케줄러만 여기서 업데이트
        if epoch < warmup_epochs and warmup_scheduler is not None:
            warmup_scheduler.step()
        
        running_loss += loss.item()
        
        # 정확도 계산 - CutMix 적용 여부에 따라 다르게 처리
        if use_cutmix:
            # 원-핫 인코딩된 레이블에서 argmax를 사용해 가장 큰 값의 인덱스 추출
            _, label_idx = labels.max(1)
        else:
            # 정수 인덱스 레이블 그대로 사용
            label_idx = labels
            
        # top-1 정확도 계산
        _, predicted = outputs.max(1)
        total += inputs.size(0)
        correct_top1 += predicted.eq(label_idx).sum().item()
        
        # top-5 정확도 계산
        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
        correct_top5 += sum([1 for i in range(len(label_idx)) if label_idx[i] in top5_idx[i]])
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy_top1, accuracy_top5

def evaluate(model, dataloader, criterion, device, epoch, phase="test"):
    """
    평가 함수 (PSKD 손실 함수 적용)
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    eval_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 원-핫 인코딩 변환 (PSKD에 필요)
            batch_size = labels.size(0)
            one_hot_labels = torch.zeros(batch_size, 100).to(device)  # CIFAR-100은 100개 클래스
            one_hot_labels.scatter_(1, labels.unsqueeze(1), 1)
            
            # 순전파
            outputs = model(inputs)
            
            # PSKD 손실 계산
            loss = criterion(outputs, one_hot_labels)
            eval_loss += loss.item()
            
            # top-1 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct_top1 += (predicted == labels).sum().item()
            
            # top-5 정확도 계산
            _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
            correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
    
    # 평균 손실 및 정확도 계산
    eval_loss = eval_loss / len(dataloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력
    print(f'{phase.capitalize()} set: Epoch: {epoch+1}, Average loss:{eval_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return eval_loss, accuracy_top1, accuracy_top5

# 메인 학습 루프
def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, num_epochs=None, patience=None, max_epochs_wait=None, warmup_scheduler=None, main_scheduler=None, warmup_epochs=None):
    """
    메인 학습 루프 (accuracy 기준 early stopping) - config에서 기본값 가져오기
    """
    # config에서 값 가져오기 (None이면)
    if num_epochs is None:
        num_epochs = config["num_epochs"]
    if patience is None:
        patience = config["patience"]
    if max_epochs_wait is None:
        max_epochs_wait = config["max_epochs_wait"]
    if warmup_epochs is None:
        warmup_epochs = config["warmup_epochs"]
        
    # 정확도 기반 얼리 스토핑 사용
    early_stopping = AccuracyEarlyStopping(patience=patience, verbose=True, path='checkpoint.pt', max_epochs=max_epochs_wait)
    
    best_test_acc_top1 = 0.0
    best_test_acc_top5 = 0.0
    
    # 테스트 정확도 기록을 위한 리스트
    test_acc_top1_history = []
    
    # tqdm을 사용한 진행 상황 표시
    for epoch in tqdm(range(num_epochs)):
        # 학습
        train_loss, train_acc_top1, train_acc_top5 = train(
            model, 
            trainloader, 
            criterion, 
            optimizer, 
            device, 
            epoch, 
            warmup_scheduler, 
            warmup_epochs
        )
        
        # 테스트 데이터로 평가
        test_loss, test_acc_top1, test_acc_top5 = evaluate(model, testloader, criterion, device, epoch, phase="test")

        # 웜업 이후 스케줄러 업데이트 
        if epoch >= warmup_epochs and main_scheduler is not None:
            if isinstance(main_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                main_scheduler.step(test_acc_top1)  # 테스트 정확도에 따라 학습률 업데이트
            else:
                main_scheduler.step()  # 다른 스케줄러 (예: CosineAnnealingLR)
            
        # 테스트 정확도 기록
        test_acc_top1_history.append(test_acc_top1)
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy_top1": train_acc_top1,
            "train_accuracy_top5": train_acc_top5,
            "test_loss": test_loss,
            "test_accuracy_top1": test_acc_top1,
            "test_accuracy_top5": test_acc_top5
        })
            
        # 최고 정확도 모델 저장 (top-1 기준)
        if test_acc_top1 > best_test_acc_top1:
            best_test_acc_top1 = test_acc_top1
            best_test_acc_top5_at_best_top1 = test_acc_top5
            print(f'새로운 최고 top-1 정확도: {best_test_acc_top1:.2f}%, top-5 정확도: {best_test_acc_top5_at_best_top1:.2f}%')
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
        
        # top-5 accuracy 기록 업데이트
        if test_acc_top5 > best_test_acc_top5:
            best_test_acc_top5 = test_acc_top5
            print(f'새로운 최고 top-5 정확도: {best_test_acc_top5:.2f}%')

        # Early stopping 체크 (test_acc_top1 기준)
        early_stopping(test_acc_top1, model, epoch)
        if early_stopping.early_stop:
            print(f"에폭 {epoch+1}에서 학습 조기 종료. 최고 성능 에폭: {early_stopping.best_epoch+1}")
            break
    
    # 훈련 완료 후 최고 모델 로드
    print("테스트 정확도 기준 최고 모델 로드 중...")
    model_path = f'best_model_{wandb.run.name}.pth'
    model.load_state_dict(torch.load(model_path))

    # 최종 테스트 세트 평가
    final_test_loss, final_test_acc_top1, final_test_acc_top5 = evaluate(model, testloader, criterion, device, num_epochs-1, phase="test")
    
    print(f'완료! 최고 테스트 top-1 정확도: {best_test_acc_top1:.2f}%, 최고 테스트 top-5 정확도: {best_test_acc_top5:.2f}%')
    print(f'최종 테스트 top-1 정확도: {final_test_acc_top1:.2f}%, 최종 테스트 top-5 정확도: {final_test_acc_top5:.2f}%')
    
    # WandB에 최종 결과 기록
    wandb.run.summary["best_test_accuracy_top1"] = best_test_acc_top1
    wandb.run.summary["best_test_accuracy_top5"] = best_test_acc_top5
    wandb.run.summary["final_test_accuracy_top1"] = final_test_acc_top1
    wandb.run.summary["final_test_accuracy_top5"] = final_test_acc_top5

    # Early stopping 정보 저장
    if early_stopping.early_stop:
        wandb.run.summary["early_stopped"] = True
        wandb.run.summary["early_stopped_epoch"] = epoch+1
        wandb.run.summary["best_epoch"] = early_stopping.best_epoch+1
    else:
        wandb.run.summary["early_stopped"] = False


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화 - config 기반
if config["model"] == "resnet18":
    model = resnet18().to(device)
else:
    raise ValueError(f"지원되지 않는 모델: {config['model']}")

# PSKD 손실 함수 설정 (두 번째 코드의 손실 함수 유지)
criterion = PSKD()

# 옵티마이저 설정 - ResNeXt와 동일한 파라미터 적용
if config["optimizer"] == "SAM_SGD":
    base_optimizer = optim.SGD
    optimizer = SAM(
        model.parameters(), 
        base_optimizer, 
        lr=config["learning_rate"],
        momentum=config["momentum"],
        weight_decay=config["weight_decay"],
        nesterov=config["nesterov"],
        rho=config["rho"],
        adaptive=config["adaptive"]
    )
else:
    raise ValueError(f"지원되지 않는 옵티마이저: {config['optimizer']}")

# WarmUpLR 스케줄러 초기화 - ResNeXt와 동일하게 10 에폭으로 설정
warmup_steps = config["warmup_epochs"] * len(trainloader)
warmup_scheduler = WarmUpLR(optimizer, total_iters=warmup_steps)

# 웜업 이후 사용할 스케줄러 설정 - ResNeXt와 동일한 파라미터 적용
if config["lr_scheduler"] == "ReduceLROnPlateau":
    main_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max',
        factor=config["lr_factor"],
        patience=config["lr_patience"],
        verbose=True,
        threshold=config["lr_threshold"],
        min_lr=config["min_lr"]
    )
else:
    raise ValueError(f"지원되지 않는 스케줄러: {config['lr_scheduler']}")

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속 - 여러 GPU 사용
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)

# 훈련 시작 시간 기록
start_time = time.time()

# 메인 학습 루프 호출 - 명시적인 파라미터 전달 대신 기본값(config에서 자동으로 가져옴) 사용
main_training_loop(
    model=model,
    trainloader=trainloader,
    testloader=testloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    warmup_scheduler=warmup_scheduler,
    main_scheduler=main_scheduler
)

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"전체 학습 시간: {total_time:.2f} 초")

# WandB 실행 종료
wandb.finish()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/guswls/.netrc


Files already downloaded and verified
Files already downloaded and verified
Train set size: 50000
Test set size: 10000
Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


2개의 GPU를 사용합니다.


  0%|                                                                                                       | 0/300 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Test set: Epoch: 102, Average loss:0.9667, Top-1 Accuracy: 73.0700%, Top-5 Accuracy: 93.3200%, Time consumed:9.87s

EarlyStopping 카운터: 29 / 30
Epoch [103], Batch [50/391], Loss: 0.8955, LR: 0.000001
Epoch [103], Batch [100/391], Loss: 0.7411, LR: 0.000001
Epoch [103], Batch [150/391], Loss: 0.6818, LR: 0.000001
Epoch [103], Batch [200/391], Loss: 1.9637, LR: 0.000001
Epoch [103], Batch [250/391], Loss: 3.0246, LR: 0.000001
Epoch [103], Batch [300/391], Loss: 2.3985, LR: 0.000001
Epoch [103], Batch [350/391], Loss: 2.9174, LR: 0.000001
Train set: Epoch: 103, Average loss:1.6457, LR: 0.000001 Top-1 Accuracy: 66.8320%, Top-5 Accuracy: 87.9940%, Time consumed:89.01s


 34%|█████████████████████████████▉                                                          | 102/300 [2:51:26<5:32:46, 100.84s/it]

Test set: Epoch: 103, Average loss:1.0008, Top-1 Accuracy: 72.7500%, Top-5 Accuracy: 92.9800%, Time consumed:9.50s

EarlyStopping 카운터: 30 / 30
에폭 103에서 학습 조기 종료. 최고 성능 에폭: 73
테스트 정확도 기준 최고 모델 로드 중...


Test set: Epoch: 300, Average loss:0.9563, Top-1 Accuracy: 73.2900%, Top-5 Accuracy: 93.4700%, Time consumed:10.17s

완료! 최고 테스트 top-1 정확도: 73.29%, 최고 테스트 top-5 정확도: 93.47%
최종 테스트 top-1 정확도: 73.29%, 최종 테스트 top-5 정확도: 93.47%
전체 학습 시간: 10296.29 초


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
learning_rate,▂▄▆▇██████▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy_top1,▁▁▂▃▃▅▆▆▇▇▇▇████████████████████████████
test_accuracy_top5,▁▃▄▅▆▇▇▇▇▇▇▇▇███████████████████████████
test_loss,█▆▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy_top1,▁▂▂▃▃▅▅▆▆▆▆▇▇▇▇▇▇▇█▇████████████████████
train_accuracy_top5,▁▃▄▆▆▇▇▇▇███████████████████████████████
train_loss,█▇▆▆▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,73
best_test_accuracy_top1,73.29
